## Calculating Network Error with Loss

We need to way to find how far away from the "truth" we are. For this we are going to use cross-entropy loss calculation. We will be using one-hot outputs for testing - that is, only one of the items in the list will be 1, the rest will be zero. You could say, the list has "one-hot" value. 

In [7]:
import math
import numpy as np

In [2]:

# example output from model
softmax_output = [0.7, 0.1, 0.2]
# example target
target_output = [1,0,0]

loss = -(math.log(softmax_output[0])*target_output[0] +
        math.log(softmax_output[1])*target_output[1] +
         math.log(softmax_output[2])*target_output[2]
        )

print(loss)

0.35667494393873245


Note that because the output is a one-hot output only one of the values actually is going to be valid. the rest will be zeros. Because of this we can just use the expected indices instead to pull the the confidence value - see below

In [4]:
softmax_outputs = [[0.7, 0.1, 0.2],
                   [0.1, 0.5, 0.4],
                   [0.02, 0.9, 0.08]]

# the index that we expect to be positive.
class_targets = [0, 1, 1]

# Print out confidence in the correct output
for targ_idx, distribution in zip(class_targets, softmax_outputs):
    print(distribution[targ_idx])

0.7
0.5
0.9


In [19]:
# Do the same thing with numpys
softmax_outputs = np.array(
    [[0.7, 0.1, 0.2],
     [0.1, 0.5, 0.4],
     [0.02, 0.9, 0.08]])

# the index that we expect to be positive.
class_targets = [0, 1, 1]

print(softmax_outputs[[0, 1, 2], class_targets])

[0.7 0.5 0.9]


In [9]:
print(softmax_outputs[range(len(softmax_outputs)), class_targets])

[0.7 0.5 0.9]


In [15]:
neg_log = -np.log(softmax_outputs[range(len(softmax_outputs)), class_targets])
average_loss = np.mean(neg_log)
print(average_loss)

0.38506088005216804


The classes can either be one-hot like we talked about earlier or they could be numberically classed like we did above - IE: `[0,0,1]` or `2`. Here we'll do the same as above, but with one-hot values instead of categorical values

In [22]:
softmax_outputs = np.array(
    [[0.7, 0.1, 0.2],
     [0.1, 0.5, 0.4],
     [0.02, 0.9, 0.08]])

class_targets = np.array([[1,0,0],
                          [0,1,0],
                          [0,1,0]])
if len(class_targets.shape) == 1:
    correct_confidences = softmax_outputs[
     range(len(softmax_outputs)),
     class_targets]
elif len(class_targets.shape) == 2:
    correct_confidences = np.sum(
     softmax_outputs*class_targets,
     axis=1)
    
print(correct_confidences)
neg_log = -np.log(correct_confidences)
average_loss = np.mean(neg_log)
print(average_loss)

[0.7 0.5 0.9]
0.38506088005216804


One issue that we are going to run into is if the model has 100% confidence in a value - we will then be calculated the log of 0 `log(0)` and that results in an error. We can solve this issue by clipping the values to a given range - a very small small value, but not zero, will not cause the failure. We can apply this using numpy's `clip` function. 
```python
y_pred_clipped = np.clip(y_pred, 1e-7, 1-1e-7)
```

In [23]:
np.log(0)

<ipython-input-23-f6e7c0610b57>:1: RuntimeWarning: divide by zero encountered in log
  np.log(0)


-inf

In [24]:
np.log(1e-7)

-16.11809565095832